In [1]:
!pip install "unsloth[colab] @ git+https://github.com/unslothai/unsloth.git"

  Cloning https://github.com/unslothai/unsloth.git to /tmp/pip-install-13cmbfrr/unsloth_5246270037484a6c9b6d2601730c53e4
  Running command git clone --filter=blob:none --quiet https://github.com/unslothai/unsloth.git /tmp/pip-install-13cmbfrr/unsloth_5246270037484a6c9b6d2601730c53e4
  Resolved https://github.com/unslothai/unsloth.git to commit c1ac4d2707574868767345e76ebe49c8353f9057
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.0/105.0 MB 8.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.8/211.8 MB 2.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 507.1/507.1 kB 48.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 22.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 270.9/270.9 kB 30.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━

In [2]:
!pip install "git+https://github.com/huggingface/transformers.git"

  Cloning https://github.com/huggingface/transformers.git to /tmp/pip-req-build-abcf8m_p
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transformers.git /tmp/pip-req-build-abcf8m_p
  Resolved https://github.com/huggingface/transformers.git to commit a1668cc72e720090206ff598719a7ee6683a7417
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for transformers: filename=transformers-4.37.0.dev0-py3-none-any.whl size=8369662 sha256=ba00bca28b30071466981912147207c7e0149c858bdc696a2ad91c6b3337a03f
  Stored in directory: /tmp/pip-ephem-wheel-cache-q0qk3_0l/wheels/e7/9c/5b/e1a9c8007c343041e61cc484433d512ea9274272e3fcbe7c16
Successfully built transformers
  Attempting uninstall: transformers
    Found existing installation: transformers 4.35.2
    Uninstalling transformers-4.35.2:
      Successfully uninstalled transformers-4.35.2


In [3]:
!pip install trl

In [4]:
from unsloth import FastLanguageModel
import torch
from trl import SFTTrainer
from transformers import TrainingArguments

/usr/local/lib/python3.10/dist-packages/unsloth/__init__.py:67: UserWarning: CUDA is not linked properly.
We shall run `ldconfig /usr/lib64-nvidia` to try to fix it.
  warnings.warn(


In [5]:
from datasets import load_dataset

In [6]:
max_seq_length = 2048

In [7]:
dataset = load_dataset("imdb", split="train")

Generating train split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/25000 [00:00<?, ? examples/s]

Generating unsupervised split:   0%|          | 0/50000 [00:00<?, ? examples/s]

In [8]:
dataset


Dataset({
    features: ['text', 'label'],
    num_rows: 25000
})

In [9]:
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/mistral-7b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype=None,
    load_in_4bit=True
)

config.json:   0%|          | 0.00/1.05k [00:00<?, ?B/s]

==((====))==  Unsloth: Fast Mistral patching release 2024.1
   \\   /|    GPU: Tesla T4. Max memory: 14.748 GB
O^O/ \_/ \    CUDA capability = 7.5. Xformers = 0.0.22.post7. FA = False.
\        /    Pytorch version: 2.1.0+cu121. CUDA Toolkit = 12.1
 "-____-"     bfloat16 = FALSE. Platform = Linux

You passed `quantization_config` to `from_pretrained` but the model you're loading already has a `quantization_config` attribute. The `quantization_config` attribute will be overwritten with the one you passed to `from_pretrained`.


model.safetensors:   0%|          | 0.00/4.13G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/971 [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/438 [00:00<?, ?B/s]

In [10]:
model = FastLanguageModel.get_peft_model(
    model,
    r = 16,
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
    lora_alpha = 16,
    lora_dropout = 0,
    bias = "none",
    use_gradient_checkpointing = True,
    random_state = 3407,
    max_seq_length = max_seq_length
)

Unsloth 2024.1 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


In [11]:
trainer = SFTTrainer(
    model = model,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    tokenizer = tokenizer,
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_steps = 10,
        max_steps = 60,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 1,
        output_dir = "unsloth-test",
        optim = "adamw_8bit",
        seed = 3407,
    ),
)
trainer.train()

Map:   0%|          | 0/25000 [00:00<?, ? examples/s]

Unsloth: `use_cache=True` is incompatible with gradient checkpointing. Setting `use_cache=False`


Step,Training Loss
1,2.255200
2,2.526500
3,2.573900
4,2.392600
5,2.449800
6,2.320000
7,2.416200
8,2.568600
9,2.698400
10,2.387100


TrainOutput(global_step=60, training_loss=2.4500720421473186, metrics={'train_runtime': 675.34, 'train_samples_per_second': 0.711, 'train_steps_per_second': 0.089, 'total_flos': 9418690861547520.0, 'train_loss': 2.4500720421473186, 'epoch': 0.02})

In [13]:
inputs = tokenizer(
    [
      "I really liked the movie because it shows emotions and talks humanity."
    ],
    return_tensors="pt",
).to("cuda")

In [14]:
inputs

{'input_ids': tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723]], device='cuda:0'), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]], device='cuda:0')}

In [15]:
outputs = model.generate(**inputs, max_new_tokens=128, use_cache=True)

Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


In [16]:
outputs

tensor([[    1,   315,  1528,  8232,   272,  5994,  1096,   378,  4370, 13855,
           304, 15066, 17676, 28723,   661,   349,   264,  1215,  1179,  5994,
           304,   315,  6557,   378,   298,  3376, 28723,   661,   349,   264,
          1215,  1179,  5994,   304,   315,  6557,   378,   298,  3376, 28723,
           661,   349,   264,  1215,  1179,  5994,   304,   315,  6557,   378,
           298,  3376, 28723,   661,   349,   264,  1215,  1179,  5994,   304,
           315,  6557,   378,   298,  3376, 28723,   661,   349,   264,  1215,
          1179,  5994,   304,   315,  6557,   378,   298,  3376, 28723,   661,
           349,   264,  1215,  1179,  5994,   304,   315,  6557,   378,   298,
          3376, 28723,   661,   349,   264,  1215,  1179,  5994,   304,   315,
          6557,   378,   298,  3376, 28723,   661,   349,   264,  1215,  1179,
          5994,   304,   315,  6557,   378,   298,  3376, 28723,   661,   349,
           264,  1215,  1179,  5994,   304,   315,  

In [17]:
tokenizer.batch_decode(outputs)

['<s> I really liked the movie because it shows emotions and talks humanity. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to everyone. It is a very good movie and I recommend it to']

In [18]:
model.save_pretrained("lora_model")

In [19]:
import locale
locale.getpreferredencoding = lambda: "UTF-8"

In [20]:
!pip install huggingface_hub


In [21]:
from huggingface_hub import notebook_login

In [25]:
notebook_login()

In [27]:
# model.push_to_hub("skuma307/unsloth_4bit_mistral_imdb_model")